In [ ]:
%%capture
!pip install ipython-autotime

%load_ext autotime

time: 3.72 ms (started: 2022-05-03 17:49:31 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
time: 16.5 s (started: 2022-05-03 17:49:31 +00:00)


In [ ]:
# !mkdir 'Data'

time: 1.21 ms (started: 2022-05-03 17:49:48 +00:00)


In [ ]:
!cp -r '/content/gdrive/MyDrive/ESE_650_Project/data/' '/content/'

time: 3.64 s (started: 2022-05-03 17:49:48 +00:00)


In [ ]:
!rm -r '/content/Data'

rm: cannot remove '/content/Data': No such file or directory
time: 160 ms (started: 2022-05-03 17:49:51 +00:00)


In [ ]:
# !pip3 uninstall tensorboard
# !pip3 uninstall tensorflow

time: 1.28 ms (started: 2022-05-03 17:49:52 +00:00)


In [ ]:
# !pip uninstall -y tensorboard-plugin-wit

time: 3.2 ms (started: 2022-05-03 17:49:52 +00:00)


In [ ]:
# !pip3 install --ignore-installed tf-nightly

time: 805 µs (started: 2022-05-03 17:49:52 +00:00)


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

from tqdm import tqdm

import matplotlib.pyplot as plt

# logging
from torch.utils.tensorboard import SummaryWriter
# import wandb
# from wandb.fastai import WandbCallback
import pandas as pd

## scene loader 
import sys
import h5py
import json
import numpy as np
import random
import skimage.io
from skimage.transform import resize

import os

time: 6.14 s (started: 2022-05-03 17:49:52 +00:00)


# Seeding for reproducability

In [ ]:
def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(seed)

  return

seed_everything()

time: 22.3 ms (started: 2022-05-03 17:49:58 +00:00)


# Hyperparameters

In [ ]:
## Constants.py

# -*- coding: utf-8 -*-

LOCAL_T_MAX = 5 # repeat step size
RMSP_ALPHA = 0.99 # decay parameter for RMSProp
RMSP_EPSILON = 0.1 # epsilon parameter for RMSProp
LEARNING_RATE = 1e-2
GAMMA = 0.99 # discount factor for rewards
ENTROPY_BETA = 0.01 # entropy regurarlization constant

SCREEN_WIDTH = 84
SCREEN_HEIGHT = 84
HISTORY_LENGTH = 4

ACTION_SIZE=4

# keys are scene names, and values are a list of location ids (navigation targets)
TASK_LIST = {
  'bathroom_02'    : ['26', '37', '43', '53', '69'],
  'bedroom_04'     : ['134', '264', '320', '384', '387'],
  'kitchen_02'     : ['90', '136', '157', '207', '329'],
  'living_room_08' : ['92', '135', '193', '228', '254']
}


time: 29.3 ms (started: 2022-05-03 17:49:58 +00:00)


# Simulation environment setup

In [ ]:
## Scene Loader

# -*- coding: utf-8 -*-
class THORDiscreteEnvironment(object):

  def __init__(self, config=dict()):

    # configurations
    self.scene_name          = config.get('scene_name', 'bedroom_04')
    self.random_start        = config.get('random_start', True)
    self.n_feat_per_locaiton = config.get('n_feat_per_locaiton', 1) # 1 for no sampling
    self.terminal_state_id   = config.get('terminal_state_id', 0)

    self.h5_file_path = config.get('h5_file_path', 'data/%s.h5'%self.scene_name)
    self.h5_file      = h5py.File(self.h5_file_path, 'r')

    self.locations   = self.h5_file['location'][()]
    self.rotations   = self.h5_file['rotation'][()]
    self.n_locations = self.locations.shape[0]

    self.terminals = np.zeros(self.n_locations)
    self.terminals[self.terminal_state_id] = 1
    self.terminal_states, = np.where(self.terminals)

    self.transition_graph = self.h5_file['graph'][()]
    self.shortest_path_distances = self.h5_file['shortest_path_distance'][()]

    self.history_length = HISTORY_LENGTH
    self.screen_height  = SCREEN_HEIGHT
    self.screen_width   = SCREEN_WIDTH

    # we use pre-computed fc7 features from ResNet-50
    # self.s_t = np.zeros([self.screen_height, self.screen_width, self.history_length])
    self.s_t      = np.zeros([2048, self.history_length])
    self.s_t1     = np.zeros_like(self.s_t)
    self.s_target = self._tiled_state(self.terminal_state_id)

    self.reset()

  # public methods

  def reset(self):
    # randomize initial state
    while True:
      k = random.randrange(self.n_locations)
      min_d = np.inf
      # check if target is reachable
      for t_state in self.terminal_states:
        dist = self.shortest_path_distances[k][t_state]
        min_d = min(min_d, dist)
      # min_d = 0  if k is a terminal state
      # min_d = -1 if no terminal state is reachable from k
      if min_d > 0: break

    # reset parameters
    self.current_state_id = k
    self.s_t = self._tiled_state(self.current_state_id)

    self.reward   = 0
    self.collided = False
    self.terminal = False

  def step(self, action):
    assert not self.terminal, 'step() called in terminal state'
    k = self.current_state_id
    if self.transition_graph[k][action] != -1:
      self.current_state_id = self.transition_graph[k][action]
      if self.terminals[self.current_state_id]:
        self.terminal = True
        self.collided = False
      else:
        self.terminal = False
        self.collided = False
    else:
      self.terminal = False
      self.collided = True

    self.reward = self._reward(self.terminal, self.collided)
    self.s_t1 = np.append(self.s_t[:,1:], self.state, axis=1)

  def update(self):
    self.s_t = self.s_t1

  # private methods

  def _tiled_state(self, state_id):
    k = random.randrange(self.n_feat_per_locaiton)
    f = self.h5_file['resnet_feature'][state_id][k][:,np.newaxis]
    return np.tile(f, (1, self.history_length))

  def _reward(self, terminal, collided):
    # positive reward upon task completion
    if terminal: return 10.0
    # time penalty or collision penalty
    return -0.1 if collided else -0.01

  # properties

  @property
  def action_size(self):
    # move forward/backward, turn left/right for navigation
    return ACTION_SIZE 

  @property
  def action_definitions(self):
    action_vocab = ["MoveForward", "RotateRight", "RotateLeft", "MoveBackward"]
    return action_vocab[:ACTION_SIZE]

  @property
  def observation(self):
    return self.h5_file['observation'][self.current_state_id]

  @property
  def state(self):
    # read from hdf5 cache
    k = random.randrange(self.n_feat_per_locaiton)
    return self.h5_file['resnet_feature'][self.current_state_id][k][:,np.newaxis]

  @property
  def target(self):
    return self.s_target

  @property
  def x(self):
    return self.locations[self.current_state_id][0]

  @property
  def z(self):
    return self.locations[self.current_state_id][1]

  @property
  def r(self):
    return self.rotations[self.current_state_id]




time: 438 ms (started: 2022-05-03 17:49:58 +00:00)


In [ ]:
## load scene into environment
scene_name = 'bedroom_04'

env = THORDiscreteEnvironment({
'random_start': True,
'scene_name': scene_name,
'h5_file_path': 'data/%s.h5'%scene_name,
'terminal_state_id' : 134
})

print(env.scene_name)
print(env.terminal_state_id)

bedroom_04
134
time: 29 ms (started: 2022-05-03 17:49:58 +00:00)


# Logging Setup

In [ ]:
# train_args = {
#     'save_eval_checkpoints': False,
#     'learning_rate':1e-2, 
#     'num_train_epochs': 100, 
#     'reprocess_input_data': True, 
#     'overwrite_output_dir': True,
#     'wandb_project': "ese_650_ac_try_1",
# }

#     # 'num_train_epochs': 3, 


# # train_args = {
# #     'evaluate_during_training': True,
# #     'logging_steps': 10,
# #     'evaluate_during_training_steps': 10,
# #     'save_eval_checkpoints': False,
# #     'train_batch_size': 128,
# #     'eval_batch_size': 256,
# #     'learning_rate':1e-5, 
# #     'num_train_epochs': 1, 
# #     'reprocess_input_data': True, 
# #     'overwrite_output_dir': True,
# #     'wandb_project': "visualization-demo",
# # }

time: 17.1 ms (started: 2022-05-03 17:49:58 +00:00)


In [ ]:
# wandb.init(anonymous="allow", config=train_args, entity = 'AC model')

time: 861 µs (started: 2022-05-03 17:49:59 +00:00)


In [ ]:
!rm -rf ./runs/ 

time: 174 ms (started: 2022-05-03 17:49:59 +00:00)


In [ ]:
!mkdir 'runs'

time: 154 ms (started: 2022-05-03 17:49:59 +00:00)


In [ ]:
# !mkdir './runs/AC_model'

time: 1.12 ms (started: 2022-05-03 17:49:59 +00:00)


In [ ]:
# writer = SummaryWriter('./runs/AC_model')
writer = SummaryWriter()

time: 4.95 s (started: 2022-05-03 17:49:59 +00:00)


In [ ]:
d = {"epoch":[], "train_episode_reward":[], "train_episode_length":[], "train_value_loss":[], "train_policy_loss":[], "train_total_loss":[]}
log_df = pd.DataFrame(d)

time: 14.7 ms (started: 2022-05-03 17:50:04 +00:00)


In [ ]:
log_df

,epoch,train_episode_reward,train_episode_length,train_value_loss,train_policy_loss,train_total_loss


time: 33 ms (started: 2022-05-03 17:50:04 +00:00)


# Actor-Critic Model

In [ ]:
class MyActorCritic(nn.Module):

    def __init__(self):
        super(MyActorCritic, self).__init__()

        # generic siamese layer: <start>
        self.fc1 = nn.Linear(in_features=8192, out_features=512)
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        # generic siamese layer: <end>

        # Scene specific layer: <start> 
        # for now we have only one scene speicific layer and only one thread i.e only one target in one scene
        self.fc3 = nn.Linear(in_features=512, out_features=512)

        self.actions_fc = nn.Linear(in_features=512, out_features=4)
        self.value_fc  = nn.Linear(in_features=512, out_features=1)
        # Scene specific layer: <end>


    def forward(self, input_image_embedding, target_image_embedding):

        # generic siamese latyer
        x1 = F.relu(self.fc1(input_image_embedding))
        x2 = F.relu(self.fc1(target_image_embedding))

        x_combined = torch.cat((x1, x2)) # combine input image emb and target image emb, may need to add axis for concat later

        x = F.relu(self.fc2(x_combined))


        # scene specific layer

        x = F.relu(self.fc3(x))
        x1 = self.actions_fc(x)
        # print(f' actions values before softmax:\n{x1}\n')
        actions_prob = F.softmax(x1, dim=0)
        value = self.value_fc(x)

        return actions_prob, value


time: 58.1 ms (started: 2022-05-03 17:50:04 +00:00)


In [ ]:
num_epochs = 50000
model = MyActorCritic()
lr = LEARNING_RATE
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=RMSP_ALPHA, eps=RMSP_EPSILON)
device_name = 'cuda:0'
# device_name = 'cpu'

max_local_timestep = LOCAL_T_MAX
entropy_beta = ENTROPY_BETA

torch.autograd.set_detect_anomaly(True)

time: 163 ms (started: 2022-05-03 17:50:04 +00:00)


In [ ]:
model.to(device_name)

for epoch in tqdm(range(num_epochs)):

    epoch_loss = 0
    # model.double()
    # model.train()

    env.reset()

    states = []
    actions = []
    policy = []
    rewards = []
    values = []
    targets = []

    terminal_reached = False

    episode_length = 0
    episode_reward = 0
    local_t = 0

    for t in range(max_local_timestep):

        torch_s_t = torch.from_numpy(env.s_t).float().flatten()  # state embedding in torch convertinng from 2048 x 4, to 8192 x 1
        torch_target = torch.from_numpy(env.target).float().flatten()  # target embedding in torch

        torch_s_t = torch_s_t.to(device_name)
        torch_target = torch_target.to(device_name)

        actions_prob, value = model(torch_s_t, torch_target)

        action = None
        with torch.no_grad():
            action = torch.multinomial(actions_prob, 1).item()  # sample action according to the probability returned by the network


        # store required items
        states.append(env.s_t)
        policy.append(actions_prob)
        actions.append(action)
        values.append(value)
        targets.append(env.target)

        # process game
        env.step(action)

        # receive game result
        reward = env.reward
        terminal = env.terminal

        # ad-hoc reward for navigation
        reward = 10.0 if terminal else -0.01
        if episode_length > 5e3: terminal = True

        episode_reward += reward
        episode_length += 1
        # episode_max_q = max(episode_max_q, np.max(value_)) !!!! DO Q VALUE CLIPPING LATER !!!!!!!

        # clip and append reward
        rewards.append(np.clip(reward, -1, 1))

        local_t += 1

        # s_t1 -> s_t
        env.update()

        if terminal:
            terminal_end = True
            # sys.stdout.write("time %d | thread #%d | scene %s | target #%s\n%s %s episode reward = %.3f\n%s %s episode length = %d\n%s %s episode max Q  = %.3f\n" % (global_t, self.thread_index, self.scene_scope, self.task_scope, self.scene_scope, self.task_scope, self.episode_reward, self.scene_scope, self.task_scope, self.episode_length, self.scene_scope, self.task_scope, self.episode_max_q))

            # summary_values = {
            # "episode_reward_input": episode_reward,
            # "episode_length_input": float(episode_length),
            # # "episode_max_q_input": episode_max_q,
            # # "learning_rate_input": _anneal_learning_rate(global_t)
            # }

            # self._record_score(sess, summary_writer, summary_op, summary_placeholders,
            #                 summary_values, global_t)
            # self.episode_reward = 0
            # self.episode_length = 0
            # self.episode_max_q = -np.inf
            # self.env.reset()
            # TODO: add logging
            # print('playout finished')
            # print(f'episode length: {episode_length}')
            # print(f'episode reward: {episode_reward}')
            # print(f'episode max_q: {episode_max_q}')


            break

    writer.add_scalar("train/episode_reward", episode_reward, epoch)
    writer.add_scalar("train/episode_length", episode_length, epoch)

    torch_s_t = torch.from_numpy(env.s_t).float().flatten()  # state embedding in torch
    torch_target = torch.from_numpy(env.target).float().flatten()  # target embedding in torch

    torch_s_t = torch_s_t.to(device_name)
    torch_target = torch_target.to(device_name)

    next_state_value = 0.0
    if not terminal:
        _, next_state_value = model(torch_s_t, torch_target)
        next_state_value = next_state_value.item()

    actions.reverse()
    states.reverse()
    rewards.reverse()
    values.reverse()
    policy.reverse()

    batch_si = []
    batch_policy = []
    batch_a = []
    batch_td = []
    batch_R = []
    batch_t = []
    batch_values = []

    # compute and accmulate gradients
    for (pi, ai, ri, si, Vi, ti) in zip(policy, actions, rewards, states, values, targets):
        
        next_state_value = ri + GAMMA * next_state_value
        td = next_state_value - Vi
        a = np.zeros([ACTION_SIZE])
        a[ai] = 1

        batch_si.append(si)
        batch_policy.append(pi)
        batch_a.append(a)
        batch_td.append(td)
        batch_R.append(next_state_value)
        batch_t.append(ti)
        batch_values.append(Vi)

    # COMPUTE LOSS + BACKPROP
    # !!!!!!!! compute actor/policy loss !!!!!!!!!!!!
    batch_policy_tensor = torch.stack(batch_policy, 0)

    batch_policy_tensor = torch.clip(batch_policy_tensor, 1e-15, 1.0)
    policy_log_prob = torch.log(batch_policy_tensor)
    policy_entropy = - torch.sum(batch_policy_tensor * policy_log_prob, axis=1)


    batch_a_tensor = torch.from_numpy(np.array(batch_a, dtype=np.int64)).to(device_name)
    nllLoss = F.nll_loss(policy_log_prob, torch.argmax(batch_a_tensor, dim=1), reduce=False)

    batch_td_tensor = torch.stack(batch_td)
    policy_loss = nllLoss * batch_td_tensor - policy_entropy * entropy_beta
    policy_loss = policy_loss.sum()

    # !!!!!!!!!!!! Value loss !!!!!!!!!!!!!!
    # learning rate for critic is half of actor's
    # Equivalent to 0.5 * l2 loss
    r_tensor = torch.from_numpy(np.array(batch_R, dtype=np.float32)).to(device_name)
    value_loss = (0.5 * 0.5) * F.mse_loss(torch.stack(batch_values), r_tensor, size_average=False)
    # value_loss = (0.5 * 0.5) * F.mse_loss(torch.stack(batch_values), r_tensor)

    total_loss = value_loss + policy_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()


    writer.add_scalar("train/value_loss", value_loss.cpu().detach().item(), epoch)
    writer.add_scalar("train/policy_loss", policy_loss.cpu().detach().item(), epoch)
    writer.add_scalar("train/total_loss", total_loss.cpu().detach().item(), epoch)


    #                {"epoch","train_episode_reward":[], "train_episode_length":[], "train_value_loss":[], "train_policy_loss":[], "train_total_loss":[]}
    log_df.loc[len(log_df.index)] = [epoch, episode_reward, episode_length, value_loss.cpu().detach().item(), policy_loss.cpu().detach().item(), total_loss.cpu().detach().item()] # add row to df

  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:159: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
 82%|████████▏ | 41004/50000 [1:28:35<20:02,  7.48it/s]

In [ ]:
writer.flush()
writer.close()

In [ ]:
!ls

In [ ]:
%cd './runs'

In [ ]:
!pwd

In [ ]:
!cp -r '.' '/content/gdrive/MyDrive//ESE_650_Project/train_metrics/'

In [ ]:
%cd ..

In [ ]:
# save the model parameters
import time

timestr = time.strftime("%d_%m-%H_%M_%S")
save_train_metrics_filename = env.scene_name +'_train_metrics_epoch_' + str(num_epochs) + '_' + timestr  + '.csv'

In [ ]:
# save dataframe

log_df.to_csv(save_train_metrics_filename)

In [ ]:
!cp {save_train_metrics_filename} '/content/gdrive/MyDrive//ESE_650_Project/train_metrics/'

# Save model

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
!cp -r 'model.pth' '/content/gdrive/MyDrive/ESE_650_Project/models/'

In [ ]:
# model = torch.load('model.pth')

In [ ]:
# policy_log_prob

In [ ]:
# # print(c_nan)
# print(torch.isnan(t_img_embedding).any())

In [ ]:
# for i, k in enumerate(model.parameters()):
#     if i == 2:
#         print(k[(torch.logical_not(torch.isnan(k)))].min())
#         print()

In [ ]:
# save the model parameters
import time

timestr = time.strftime("%d_%m-%H_%M_%S")
save_model_filename = env.scene_name +'myA2C_model_wts_epoch_' + str(num_epochs) + '_' + timestr  + '.pth'
torch.save(model, save_model_filename)

In [ ]:
!cp {save_model_filename} '/content/gdrive/MyDrive//ESE_650_Project/models/'

# Visualize training results

In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir ./runs

# Evaluation

In [ ]:

# for episode in tqdm(range(NUM_EVAL_EPISODES)):
for episode in tqdm(range(5)):


    model.eval()
    env.reset()

    actions = []

    final_state = None
    inital_state_id = env.current_state_id

    terminal_reached = False

    episode_length = 0
    episode_reward = 0
    local_t = 0

    terminal = False
    inital_state_to_target_dist = env.shortest_path_distances[inital_state_id][env.terminal_state_id]
    

    while not terminal:

        torch_s_t = torch.from_numpy(env.s_t).float().flatten()  # state embedding in torch convertinng from 2048 x 4, to 8192 x 1
        torch_target = torch.from_numpy(env.target).float().flatten()  # target embedding in torch

        torch_s_t = torch_s_t.to(device_name)
        torch_target = torch_target.to(device_name)

        actions_prob, value = model(torch_s_t, torch_target)

        action = torch.argmax(actions_prob)  # choose best action

        # store required items
        actions.append(action)

        # process game
        env.step(action)

        # receive game result
        reward = env.reward
        terminal = env.terminal

        # ad-hoc reward for navigation
        reward = 10.0 if terminal else -0.01
        if episode_length > 5e3: terminal = True

        episode_reward += reward
        episode_length += 1
        # episode_max_q = max(episode_max_q, np.max(value_)) !!!! DO Q VALUE CLIPPING LATER !!!!!!!

        local_t += 1

        # s_t1 -> s_t
        env.update()

        if terminal:
            final_state = env.s_t[0]
            terminal_end = True
            break

    
    final_state_to_target_dist = env.shortest_path_distances[env.current_state_id][env.terminal_state_id]
    print(f'\nnum_iter : {episode}\nepisode_length : {episode_length}')
    print(f'inital state to target state shortest dist: {inital_state_to_target_dist}')
    # print(f'shortest dist from episode termination state to goal state: {dist}')
    print(f'final state to target state shortest dist: {final_state_to_target_dist}')
    print()

    